In [29]:
#Path related stuff
import os

#Math stuff
import numpy as np
import random

#Environment stuff
import gym
import mujoco_py

#3D Object stuff
import pyvista as pv

#Pytorch Stuff 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Normal
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim import NAdam

#To import other notebooks
import import_ipynb

#Matplotlib stuff
#import matplotlib.pyplot as plt
#Helper class imports
from Helper_class2 import ThreeD_Obstacle
from Helper_class2 import Features
from Helper_class2 import AgentUtils


In [30]:
class ReplayBuffer:
  def __init__(self, buffer_size, batch_size, seed = 4):
    self.buffer = []
    self.max_size = buffer_size
    self.batch_size = batch_size
    self.random_generator = np.random.RandomState(seed)

  def append(self, state, action, reward, terminal, next_state, cost):
    if(len(self.buffer) == self.max_size):
      del self.buffer[0]

    self.buffer.append([state, action, reward, terminal, next_state, cost])

  def sample(self):
    batch = random.sample(self.buffer, self.batch_size)

    state, action, reward, terminal, next_state, cost = map(np.stack, zip(*batch))
    
    return state, action, reward, terminal, next_state, cost

  def get_buffer_size(self):
    return len(self.buffer)


In [31]:
LOG_SIG_MAX = 2
LOG_SIG_MIN = -20
epsilon = 1e-6

#Initialize Policy weights
def weights_init_(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight, gain=1)
        torch.nn.init.constant_(m.bias, 0)

class QNetwork(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_dim):
        super(QNetwork, self).__init__()
        
        # Q1 architecture
        self.linear1 = nn.Linear(num_inputs + num_actions, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, hidden_dim)
        self.linear3 = nn.Linear(hidden_dim, hidden_dim)
        self.linear4 = nn.Linear(hidden_dim, 1)

        # Q2 architecture
        self.linear5 = nn.Linear(num_inputs + num_actions, hidden_dim)
        self.linear6 = nn.Linear(hidden_dim, hidden_dim)
        self.linear7 = nn.Linear(hidden_dim, hidden_dim)
        self.linear8 = nn.Linear(hidden_dim, 1)

        self.apply(weights_init_)

    def forward(self, state, action):
        xu = torch.cat([state, action], 1)
        
        x1 = F.relu(self.linear1(xu))
        x1 = F.relu(self.linear2(x1))
        x1 = F.relu(self.linear3(x1))
        
        x1 = self.linear4(x1)

        x2 = F.relu(self.linear5(xu))
        x2 = F.relu(self.linear6(x2))
        x2 = F.relu(self.linear7(x2))
        x2 = self.linear8(x2)

        return x1, x2


class GaussianPolicy(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_dim, action_range=None):
        super(GaussianPolicy, self).__init__()
        
        self.linear1 = nn.Linear(num_inputs, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, hidden_dim)
        self.linear3 = nn.Linear(hidden_dim, hidden_dim)

        self.mean_linear = nn.Linear(hidden_dim, num_actions)
        self.log_std_linear = nn.Linear(hidden_dim, num_actions)

        self.apply(weights_init_)

        # action rescaling
        if action_range is None:
            self.action_scale = torch.tensor(1.)
            self.action_bias = torch.tensor(0.)
        else:
            self.action_scale = torch.FloatTensor([
                (np.max(action_range) - np.min(action_range)) / 2.
                ])
            self.action_bias = torch.FloatTensor([
                (np.max(action_range) - np.max(action_range)) / 2.
                ])

    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))

        mean = self.mean_linear(x)
        log_std = self.log_std_linear(x)
        log_std = torch.clamp(log_std, min=LOG_SIG_MIN, max=LOG_SIG_MAX)
        return mean, log_std

    def sample(self, state):
        mean, log_std = self.forward(state)
        std = log_std.exp()
        normal = Normal(mean, std)
        x_t = normal.rsample()  # for reparameterization trick (mean + std * N(0,1))
        y_t = torch.tanh(x_t)
        action = y_t * self.action_scale + self.action_bias
        log_prob = normal.log_prob(x_t)
        # Enforcing Action Bound
        log_prob -= torch.log(self.action_scale * (1 - y_t.pow(2)) + epsilon)
        log_prob = log_prob.sum(1, keepdim=True)
        
        mean = torch.tanh(mean) * self.action_scale + self.action_bias
        return action, log_prob, mean

    def to(self, device):
        self.action_scale = self.action_scale.to(device)
        self.action_bias = self.action_bias.to(device)
        return super(GaussianPolicy, self).to(device)

In [32]:
class SAC_Agent(object):
    def __init__(self,observation_space, action_space, gamma, lr,  tau, alpha, hidden_network_size, 
                 action_range, batch_size, agent_util):
        self.agent_util = agent_util

        self.REPLAY_BATCH_SIZE = batch_size
        self.replay_buffer = ReplayBuffer(300000, self.REPLAY_BATCH_SIZE)

        self.gamma = gamma
        self.lr = lr
        self.tau = tau
        self.alpha = alpha

        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        
        self.LEARNING_RATE_LAM = 0.001
        self.lamb = torch.tensor(-10.0, requires_grad=True, device=self.device)
        self.lamb_optimiser = torch.optim.Adam([self.lamb], lr=self.LEARNING_RATE_LAM, weight_decay=1e-5)

        self.cost_lim = 0.001

        self.iterator = 1

        self.critic = QNetwork(observation_space[0], action_space[0], hidden_network_size).to(device=self.device)
        self.critic_optim = Adam(self.critic.parameters(), lr=self.lr)

        self.critic_target = QNetwork(observation_space[0], action_space[0], hidden_network_size).to(self.device)
        

        self.critic_cost = QNetwork(observation_space[0], action_space[0], hidden_network_size).to(device=self.device)
        self.critic_cost_optim = Adam(self.critic_cost.parameters(), lr=self.lr)
        
        self.critic_target_cost = QNetwork(observation_space[0], action_space[0], hidden_network_size).to(self.device)
        
        #perfoming a soft copy of the parameters
        for target_param, param in zip( self.critic_target.parameters(), self.critic.parameters()):
            target_param.data.copy_( param.data * self.tau + (1-self.tau) * target_param.data)

        for target_param, param in zip( self.critic_target_cost.parameters(), self.critic_cost.parameters()):
            target_param.data.copy_( param.data * self.tau + (1-self.tau) * target_param.data)


        self.policy = GaussianPolicy(observation_space[0], action_space[0], hidden_network_size, action_range).to(self.device)
        self.policy_optim = Adam(self.policy.parameters(), lr=self.lr)


        # self.target_entropy = -torch.prod(torch.Tensor(action_space).to(self.device)).item()
        # self.log_alpha = torch.zeros(1, requires_grad=True, device=self.device)
        # self.alpha_optim = NAdam([self.log_alpha], lr=self.lr)


    def select_action(self, state, evaluate=False):
        state = torch.FloatTensor(state).to(self.device).unsqueeze(0)
        if evaluate is False:
            action, _, _ = self.policy.sample(state)

        return action.detach().cpu().numpy()[0]
    
    
    def gradient_clipping(self, model_parameters, clip_val):
        torch.nn.utils.clip_grad_norm_(model_parameters, max_norm=clip_val, norm_type=2)

    
    def update_parameters(self):
        # Sample a batch from memory

        state_batch, action_batch, reward_batch, mask_batch, next_state_batch, cost_batch = self.replay_buffer.sample()

        state_batch = torch.FloatTensor(state_batch).to(self.device)
        next_state_batch = torch.FloatTensor(next_state_batch).to(self.device)
        action_batch = torch.FloatTensor(action_batch).to(self.device)
        reward_batch = torch.FloatTensor(reward_batch).to(self.device).unsqueeze(1)
        mask_batch = torch.FloatTensor(mask_batch).to(self.device).unsqueeze(1)
        cost_batch = torch.FloatTensor(cost_batch).to(self.device).unsqueeze(1)

        with torch.no_grad():
            next_state_action, next_state_log_pi, _ = self.policy.sample(next_state_batch)
            
            qf1_next_target, qf2_next_target = self.critic_target(next_state_batch, next_state_action)
            min_qf_next_target = torch.min(qf1_next_target, qf2_next_target) - self.alpha * next_state_log_pi
            next_q_value = (reward_batch) + (1-mask_batch) * self.gamma * (min_qf_next_target)

            qf1_next_target_cost, _ = self.critic_target_cost(next_state_batch, next_state_action)
            #min_qf_next_target_cost = torch.min(qf1_next_target_cost, qf2_next_target_cost)
            next_q_value_cost = cost_batch + (1-mask_batch) * self.gamma * (qf1_next_target_cost)
       

        qf1, qf2 = self.critic(state_batch, action_batch) # Two Q-functions to mitigate positive bias in the policy improvement step
        qf1_c, _ = self.critic_cost(state_batch, action_batch) # Two Q-functions to mitigate positive bias in the policy improvement step

        qf1_loss = F.mse_loss(qf1, next_q_value)
        qf2_loss = F.mse_loss(qf2, next_q_value)

        qf1_cost_loss = F.mse_loss(qf1_c, next_q_value_cost)  # JQ = 𝔼(st,at)~D[0.5(Q1(st,at) - r(st,at) - γ(𝔼st+1~p[V(st+1)]))^2]


        qf_loss = (qf1_loss + qf2_loss).mean()
        qf_cost_loss = qf1_cost_loss.mean()


        self.critic_optim.zero_grad()
        qf_loss.backward()
        self.critic_optim.step()

        self.critic_cost_optim.zero_grad()
        qf_cost_loss.backward()
        self.critic_cost_optim.step()


        for target_param, param in zip( self.critic_target.parameters(), self.critic.parameters()):
            target_param.data.copy_( param.data * self.tau + (1-self.tau) * target_param.data)
        
        for target_param, param in zip( self.critic_target_cost.parameters(), self.critic_cost.parameters()):
            target_param.data.copy_( param.data * self.tau + (1-self.tau) * target_param.data)


        pi, log_pi, _ = self.policy.sample(state_batch)
        

        qf1_pi, qf2_pi = self.critic(state_batch, pi)
        min_qf_pi = torch.min(qf1_pi, qf2_pi)
        first_term = (self.alpha * log_pi) - min_qf_pi

        qf1_pi_c, _ = self.critic_cost(state_batch, pi)
        
        second_term = 10*self.lamb * qf1_pi_c

        policy_loss = (first_term + second_term).mean()# Jπ = 𝔼st∼D,εt∼N[α * logπ(f(εt;st)|st) − Q(st,f(εt;st))]


        self.policy_optim.zero_grad()
        policy_loss.backward()
        self.policy_optim.step()


        #Training lambda
        #Training lambda
        self.iterator += 1
        
        if self.iterator % 3 == 0:

            violation = cost_batch - self.cost_lim
            
            self.log_lam = torch.nn.functional.softplus(self.lamb, beta=0.5)
            lambda_loss = -self.log_lam*(violation).detach()
            lambda_loss = lambda_loss.sum()

            self.lamb_optimiser.zero_grad()
            lambda_loss.backward()
            self.lamb_optimiser.step()

            self.iterator = 1

        
        return qf1_loss.item(), qf2_loss.item(), policy_loss.item()



In [33]:
mj_path = mujoco_py.utils.discover_mujoco()
xml_path = os.path.join(mj_path, 'model', 'ur5.xml')
model = mujoco_py.load_model_from_path(xml_path)
sim = mujoco_py.MjSim(model)
viewer = mujoco_py.MjRenderContextOffscreen(sim, -1)

##agent.critic.state_dict()

In [34]:
def models_performance_tracker(seed, network_size, q1_avg_loss, q2_avg_loss, policy_avg_loss, avg_reward, steps, crashes, total_violations, PATH):
    torch.save({
            'seed': seed,
            'network_size': network_size,
            'q1_avg_loss': q1_avg_loss,
            'q2_avg_loss': q2_avg_loss,
            'policy_avg_loss': policy_avg_loss,
            'avg_reward': avg_reward,
            'steps': steps,
            'crashes': crashes,
            'violations': total_violations,
            #'model_state_dict': agent.critic.state_dict(),
            #'optimizer_state_dict': optimizer.state_dict(),
            }, PATH)
    


In [35]:
EPOCHS = 10000
TARGET_POS = [-0.11034687, -0.59504954, 0.2978785 ] #based on [0, -0.22, 1.67, 0.66, 1.57, 0.22]

SEEDs = [0]
NETWORKs_SEARCH = [128] #This was 128 with seed 4,5 when it was working

#Object stuff
object_position1 = [-0.11034687, -0.59504954, 0.2978785 + 0.2] #the plus is the axis radius, to be on the same plne

# layer1_shere_semi_axis = [0.17,0.17,0.17]
# layer2_shere_semi_axis = [0.35,0.35,0.35]
l1_radius = 0.35
l2_radius = 0.34

threeD_object1 = ThreeD_Obstacle(simulation = sim, sphere_or_ellipsoid= "Sphere", 
                                 radius_or_axis = 0.2 , my_position = object_position1,
                                 safety_l1 = l1_radius,
                                 safety_l2 = l2_radius,
                                 target_pos = TARGET_POS,
                                   translation_rate= [0.008, 0, 0])

initial_action = [-2.42, -1.57, -0.597, -1.48, 3.14, -1.42]

#Arm stuff
arm_utils = AgentUtils(sim, threeD_object1, initial_action, TARGET_POS) 
arm_feature_object = Features(sim, threeD_object1, arm_utils)

for seed in SEEDs:
    alpha = 0.05
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    for network in NETWORKs_SEARCH:

        agent = SAC_Agent((48,), (6,), 0.99, 0.0001, 0.005, alpha, network, 
                          [-0.0698132, 0.0698132], 256, arm_utils)
        
        q1_loss_avgs = np.array([])
        q2_loss_avgs = np.array([])
        policy_loss_avgs = np.array([])
        average_rewards = np.array([])
        steps = np.array([])
        crashes = np.array([])
        total_violations = np.array([])
        lambda_episodes = []
        for i in range(EPOCHS):
            done = 0
            distance = 0
            step = 1
            minus_on_reward = 0
            
            #storing data
            q1_loss = np.array([])
            q2_loss = np.array([])
            policy_loss = np.array([])
            rewards = np.array([])
            violations = np.array([])
            crashed = 0

            #taking the initial step
            sim.data.qpos[:] = initial_action
            sim.step()

            #Getting features of the arm robot.
            calculated_features = arm_feature_object.get_all_features().copy()
            observation = calculated_features
            
            lambda_step = []
            
            time_step_T = 5000
            for _ in range(time_step_T):
                
                action = agent.select_action(observation).copy()
                
                # current_qpos = sim.data.qpos.copy()
                # new_qpos = current_qpos + action
                sim.data.qpos[:] = arm_utils.get_joint_angles(action)


                try:
                    sim.step()
                except mujoco_py.MujocoException as e:
                    if "Got MuJoCo Warning: Nan, Inf or huge value in QACC" in str(e):
                        print("Simulation is unstable. Taking corrective action...")
                        # Do something to correct the instability, e.g. reset the simulation or adjust the control parameters
                        sim.data.qpos[:] = [-2.42, -1.57, -0.597, -1.48, 3.14, -1.41]
                        sim.step() #required to update the joints and EE
                    else:
                        # Handle other MujocoExceptions as needed
                        print(f"Unexpected MujocoException: {e}")

              
                cost = threeD_object1.compute_link_distance_to_object_when_in_l1_by_closest_points_for_cost()
                violations = np.append(violations,cost)
                
                if( threeD_object1.object_intruded().sum() >= 1):
                    done = 1
                    crashed = 1
                    #max_distance_cost = np.abs(rewards).sum()
                    #threeD_object1.reset()

                    print("crashed")

                reward = arm_utils.get_reward()

                #translate the object
                threeD_object1.linearly_translate_object_back_and_forward()

                #Getting features of the arm robot.
                calculated_features = arm_feature_object.get_all_features().copy()
                new_observation = calculated_features

                if(arm_utils.check_EE_is_in_the_target_area() == 1):#function returns a 1 if the EE coordinates are in the sphere, else 0
                    done = 1

                rewards = np.append(rewards,reward)
                

                agent.replay_buffer.append(observation, action, reward, done, new_observation, cost)
                

                observation = new_observation

                if( agent.replay_buffer.get_buffer_size() > agent.REPLAY_BATCH_SIZE ):
                    q1_l, q2_l, policy_l = agent.update_parameters()

                    q1_loss = np.append(q1_loss, q1_l)
                    q2_loss = np.append(q2_loss, q2_l)
                    policy_loss = np.append(policy_loss, policy_l)
                   

                if(done == 1):
                    arm_utils = AgentUtils(sim, threeD_object1, initial_action, TARGET_POS) #resetting the object for the initial position
                    break
                
                step+=1

            print("epoch: ", i, ",  step : ",step, ",  avg_reward : ", rewards.mean())

            q1_loss_avgs = np.append(q1_loss_avgs, q1_loss.mean())
            q2_loss_avgs = np.append(q2_loss_avgs, q2_loss.mean())
            policy_loss_avgs = np.append(policy_loss_avgs, policy_loss.mean())
            average_rewards = np.append(average_rewards, rewards.mean())
            steps = np.append(steps, step)
            crashes = np.append(crashes,crashed)
            total_violations = np.append(total_violations,violations.sum())
            
            

        name = "FinalNonScalarizedLagrange_alpha="+str(alpha) +"_cost=0.001_seed=:"+str(seed)+".pt"
        Path = "/home/padjei/Desktop/Models/Reviewers/" + name  
        
        models_performance_tracker(seed=seed, network_size=network, q1_avg_loss=q1_loss_avgs, q2_avg_loss=q2_loss_avgs, 
                                   policy_avg_loss=policy_loss_avgs, avg_reward = average_rewards, steps=steps, crashes=crashes, 
                                    total_violations=total_violations, PATH=Path)




crashed
epoch:  0 ,  step :  415 ,  avg_reward :  -1.0690160249010614
Simulation is unstable. Taking corrective action...
crashed
epoch:  1 ,  step :  418 ,  avg_reward :  -0.8570927682444327
crashed
epoch:  2 ,  step :  3112 ,  avg_reward :  -0.7832154729588328
epoch:  3 ,  step :  5001 ,  avg_reward :  -0.7884148356847998
epoch:  4 ,  step :  5001 ,  avg_reward :  -0.8200029931831372
epoch:  5 ,  step :  5001 ,  avg_reward :  -0.8337648318684553
epoch:  6 ,  step :  5001 ,  avg_reward :  -0.7414235195631413
intersected
epoch:  7 ,  step :  1566 ,  avg_reward :  -0.9423474694294358
crashed
epoch:  8 ,  step :  1711 ,  avg_reward :  -0.8463263661195726
crashed
epoch:  9 ,  step :  1738 ,  avg_reward :  -0.8285121302921565
crashed
epoch:  10 ,  step :  2334 ,  avg_reward :  -0.8003291637342239
intersected
epoch:  11 ,  step :  672 ,  avg_reward :  -0.9347289757363352
intersected
epoch:  12 ,  step :  576 ,  avg_reward :  -0.9788792618122003
crashed
epoch:  13 ,  step :  3202 ,  avg_rewa

: 